In [ ]:

# --- Step 1: Install Dependencies ---
!pip install -q transformers accelerate datasets peft trl bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.3 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [ ]:
# --- Step 2: Login to Hugging Face ---
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# --- Step 3: Setup Config ---
import os

# Change these values for your project
BASE_MODEL = "unsloth/llama-3-8b-Instruct-bnb-4bit"  # Base model
OUTPUT_DIR = "llama-finetuned"                       # Where to save adapter
HF_REPO = "rayymaxx/DirectEd-AI-LoRA"            # Your HF repo for adapter
HF_DATASET_REPO = "rayymaxx/DirectEd-AI-LoRA-Dataset"     # Your HF repo for dataset
DATASET_FILE = "dataset.jsonl"                       # Local dataset file

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# conversational_dataset.py
import json
import random

# ----------------------------
# Expanded Seed Examples
# ----------------------------
seed_examples = [
    # --- Fullstack (MERN) ---
    {
        "question": "Explain the difference between MongoDB and SQL databases.",
        "answers": [
            "MongoDB is a NoSQL database storing JSON-like documents, while SQL databases use structured tables.",
            "MongoDB is flexible with dynamic schemas; SQL enforces strict relational schemas.",
            "Think of MongoDB as flexible and SQL as structured for reliability."
        ]
    },
    {
        "question": "What is React used for in web development?",
        "answers": [
            "React is a JavaScript library for building user interfaces.",
            "It allows developers to create reusable and interactive front-end components.",
            "React helps build responsive single-page applications efficiently."
        ]
    },

    # --- Generative AI ---
    {
        "question": "What is a GAN (Generative Adversarial Network)?",
        "answers": [
            "A GAN uses a generator to create data and a discriminator to evaluate it.",
            "GANs generate realistic outputs by having two networks compete.",
            "They learn data distributions through adversarial training."
        ]
    },
    {
        "question": "Explain embeddings in AI.",
        "answers": [
            "Embeddings convert data into numeric vectors that capture meaning.",
            "They map words or images into a high-dimensional space for similarity search.",
            "Think of embeddings as coordinates where related concepts are close together."
        ]
    },

    # --- UI/UX ---
    {
        "question": "Why is user testing important in UI/UX?",
        "answers": [
            "User testing ensures the interface meets real-world needs.",
            "It reveals usability issues before launch.",
            "Testing with users helps improve the overall experience."
        ]
    },
    {
        "question": "What is the purpose of wireframing?",
        "answers": [
            "Wireframes outline layout and structure before design.",
            "They act as blueprints for UI and interactions.",
            "Wireframes help teams align on functionality early."
        ]
    },

    # --- Cloud & DevOps ---
    {
        "question": "What is Docker used for?",
        "answers": [
            "Docker packages applications into containers for consistent environments.",
            "It simplifies deployment by bundling code and dependencies together.",
            "Think of Docker as a lightweight VM for running apps anywhere."
        ]
    },
    {
        "question": "What is Kubernetes?",
        "answers": [
            "Kubernetes is an orchestration system for managing containerized applications.",
            "It automates scaling, deployment, and management of containers.",
            "Kubernetes helps run containers reliably across clusters of machines."
        ]
    },

    # --- Python Basics ---
    {
        "question": "What is a Python function?",
        "answers": [
            "A function in Python is a reusable block of code defined with def.",
            "Functions allow you to organize code into callable procedures.",
            "They take inputs, perform actions, and can return outputs."
        ]
    },
    {
        "question": "Explain Python lists.",
        "answers": [
            "A list in Python is an ordered, mutable collection of items.",
            "Lists can store mixed data types and are defined with square brackets [].",
            "They support indexing, slicing, and many built-in methods."
        ]
    },

    # --- Data Science ---
    {
        "question": "What is Pandas used for in Python?",
        "answers": [
            "Pandas is a library for working with structured data like tables.",
            "It provides DataFrames for handling rows and columns efficiently.",
            "Pandas simplifies data cleaning, analysis, and manipulation."
        ]
    },
    {
        "question": "What is NumPy?",
        "answers": [
            "NumPy is a Python library for fast numerical computing.",
            "It provides arrays and mathematical operations for scientific computing.",
            "NumPy underlies many ML and data science libraries."
        ]
    },

    # --- Algorithms & DSA ---
    {
        "question": "What is the difference between linear and binary search?",
        "answers": [
            "Linear search checks each element one by one; binary search divides the list and eliminates halves.",
            "Binary search is faster (O(log n)) but requires sorted input; linear search is O(n).",
            "Use linear search for small/unsorted data; binary for large/sorted data."
        ]
    },
    {
        "question": "Explain Big-O notation.",
        "answers": [
            "Big-O describes the worst-case time complexity of an algorithm.",
            "It expresses how runtime grows with input size.",
            "For example, O(n) grows linearly, while O(log n) grows much slower."
        ]
    },
]

# ----------------------------
# Chit-chat pool (small %)
# ----------------------------
chit_chat_examples = [
    {"q": "Hi there!", "a": "Hello! How are you doing today?"},
    {"q": "How are you?", "a": "I’m doing great, thanks for asking. How about you?"},
    {"q": "Tell me a joke.", "a": "Why do programmers prefer dark mode? Because light attracts bugs."},
    {"q": "Can you help me study?", "a": "Of course! What topic are you working on?"},
    {"q": "Thanks!", "a": "You’re welcome, happy to help anytime!"},
]

# ----------------------------
# Generation logic
# ----------------------------
synthetic_data = []
target_size = 2000  # ~2k samples (Colab safe)
types = ["qa", "quiz", "flashcard"]

def make_example(seed, ex_type):
    q = seed["question"]
    ans = random.choice(seed["answers"])

    if ex_type == "qa":
        return f"Human: {q}\n\nAssistant: {ans}"

    elif ex_type == "quiz":
        return (
            f"Human: Create a quiz about: {q}\n\nAssistant:\n"
            f"Q: {q}\n"
            f"A) Incorrect option\n"
            f"B) {ans}\n"
            f"C) Another wrong option\n"
            f"Correct Answer: B"
        )

    elif ex_type == "flashcard":
        return (
            f"Human: Make a flashcard about: {q}\n\nAssistant:\n"
            f"Front: {q}\n"
            f"Back: {ans}"
        )

def make_chitchat():
    ex = random.choice(chit_chat_examples)
    return f"Human: {ex['q']}\n\nAssistant: {ex['a']}"

# Cycle until target size
while len(synthetic_data) < target_size:
    if random.random() < 0.08:  # ~8% chit-chat
        synthetic_data.append({"text": make_chitchat()})
    else:
        seed = random.choice(seed_examples)
        ex_type = random.choice(types)
        synthetic_data.append({"text": make_example(seed, ex_type)})

# ----------------------------
# Save dataset
# ----------------------------
dataset_file = "conversational_dataset.jsonl"
with open(dataset_file, "w", encoding="utf-8") as f:
    for entry in synthetic_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ Conversational dataset created with {len(synthetic_data)} entries at {dataset_file}")


✅ Structured dataset created with 240 entries at dataset.jsonl


In [ ]:
# --- Step 5: Upload Dataset to Hugging Face Hub ---
from datasets import load_dataset

dataset = load_dataset("json", data_files=DATASET_FILE, split="train")
dataset.push_to_hub(HF_DATASET_REPO)

print(f"✅ Dataset pushed to Hugging Face Hub at {HF_DATASET_REPO}")

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########| 2.30kB / 2.30kB            

README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

✅ Dataset pushed to Hugging Face Hub at rayymaxx/DirectEd-AI-LoRA-Dataset


In [ ]:
# --- Step 6: Load Tokenizer Only (model will be loaded in Step 8) ---
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("✅ Tokenizer ready")


✅ Tokenizer ready


In [ ]:
# --- Step 7: Configure LoRA ---
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

In [ ]:


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig, pipeline
from trl import SFTTrainer
from peft import PeftModel, LoraConfig
from datasets import load_dataset
from huggingface_hub import HfApi

OUTPUT_DIR = "llama-finetuned"
BASE_MODEL = "unsloth/llama-3-8b-Instruct-bnb-4bit"
HF_REPO = "rayymaxx/DirectEd-AI-LoRA"
HF_DATASET_REPO = "rayymaxx/DirectEd-AI-LoRA-Dataset"
DATASET_FILE = "dataset.jsonl"
MAX_LEN = 512


dataset = load_dataset("json", data_files=DATASET_FILE)["train"]
print(f"✅ Loaded {len(dataset)} samples from local file")


tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])
print("🧹 Tokenized dataset")


peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# --- Load 4-bit model with offloading and memory optimization ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",             # automatically chooses GPU or CPU
    offload_folder="offload",      # use CPU offload if GPU memory is low
    offload_state_dict=True,
    torch_dtype=torch.float16
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- Training args ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,      # small batch for low memory
    gradient_accumulation_steps=4,      # effective batch size 8x1=8
    learning_rate=1e-4,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,                          # mixed precision
    gradient_checkpointing=True,        # save memory
    push_to_hub=True,
    hub_model_id=HF_REPO
)

# --- Trainer ---
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator,
    peft_config=peft_config
)

# --- Train with automatic resume ---
import os
last_checkpoint = None
if os.path.isdir(OUTPUT_DIR) and any(f.startswith("checkpoint") for f in os.listdir(OUTPUT_DIR)):
    last_checkpoint = max([os.path.join(OUTPUT_DIR, f) for f in os.listdir(OUTPUT_DIR) if f.startswith("checkpoint")], key=os.path.getmtime)
    print(f"🔁 Resuming from checkpoint: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)

# --- Save locally & push adapter ---
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
trainer.push_to_hub()
print(f"✅ Adapter saved and pushed to Hugging Face Hub: {HF_REPO}")

# --- Inference test ---
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

prompt = "Human: Explain the purpose of monitoring in LLMOps.\n\nAssistant:"
out = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)
print("✅ Model output:\n", out[0]["generated_text"])


✅ Loaded 240 samples from local file


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

🧹 Tokenized dataset


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
# --- Step 9: Save Adapter ---
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Adapter saved locally at {OUTPUT_DIR}")

In [ ]:
# --- Step 10: Push Adapter to Hugging Face Hub ---
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path=OUTPUT_DIR,
    repo_id=HF_REPO,
    repo_type="model"
)

print(f"✅ Adapter pushed to Hugging Face Hub at {HF_REPO}")

In [ ]:
# --- Step 11: Inference Test ---
from transformers import pipeline
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

prompt = "Human: Explain the purpose of monitoring in LLMOps.\n\nAssistant:"
out = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)
print("✅ Model output:\n", out[0]["generated_text"])